In [1]:
using LinearAlgebra
using Jacobi
using Test

# Prime Basis

The prime basis is defined in reference element $\hat{E}=[-1,1]^3$

#### Enrichment Vectors
In BDDF 1987 paper, they considered 6 enrichments for cubic element which for linear case are given as
$$
\hat{S}_1 = \textbf{curl} \{
	\begin{bmatrix} 0 \\ 0 \\ \hat{x}\hat{y}\hat{z} \end{bmatrix},
    \begin{bmatrix} 0 \\ 0 \\ \hat{x}\hat{y}^2 \end{bmatrix},
	\begin{bmatrix} \hat{x}\hat{y}\hat{z} \\ 0 \\ 0 \end{bmatrix}, 
	\begin{bmatrix} \hat{y}\hat{z}^2 \\ 0 \\ 0 \end{bmatrix},
	\begin{bmatrix} 0 \\ \hat{x}\hat{y}\hat{z} \\ 0 \end{bmatrix},
    \begin{bmatrix} 0 \\ \hat{x}^2\hat{z} \\ 0 \end{bmatrix}
	\}
$$

In 2012, Wheeler, Xue and Yotov added another 6 enrichments to make the dimension of prime basis 24. They defined
$$
\hat{S}_2 = \textbf{curl} \{
	\begin{bmatrix} 0 \\ 0 \\ \hat{x}^2\hat{z} \end{bmatrix},
    \begin{bmatrix} 0 \\ 0 \\ \hat{x}^2\hat{y}\hat{z}\end{bmatrix},
	\begin{bmatrix} \hat{x}\hat{y}^2 \\ 0 \\ 0 \end{bmatrix}, 
	\begin{bmatrix} \hat{x}\hat{y}^2\hat{z} \\ 0 \\ 0 \end{bmatrix},
	\begin{bmatrix} 0 \\ \hat{y}\hat{z}^{2} \\ 0 \end{bmatrix},
    \begin{bmatrix} 0 \\ \hat{x}\hat{y}\hat{z}^2 \\ 0 \end{bmatrix}
	\}
$$

Our prime basis for Hex element is

$$
\hat{\boldsymbol{V}}(\hat{E}) = P_1(\hat{E})^3 + \, \hat{S}_1 + \, \hat{S}_2
$$

In [2]:
function PrimeBasis(xhat, yhat, zhat)

    m = length(xhat)

    P = zeros(3,m,24)
    P[1,:,1] = ones(m)
    P[1,:,2] = xhat
    P[1,:,3] = yhat
    P[1,:,4] = zhat
    P[2,:,5] = ones(m)
    P[2,:,6] = xhat
    P[2,:,7] = yhat
    P[2,:,8] = zhat
    P[3,:,9] = ones(m)
    P[3,:,10] = xhat
    P[3,:,11] = yhat
    P[3,:,12] = zhat
    
    # supplement (curl term)
    P[1,:,13] = xhat .* zhat
    P[2,:,13] = -yhat .* zhat
    P[1,:,14] = 2*xhat .* yhat
    P[2,:,14] = -yhat.^2
    P[2,:,15] = -2*xhat .* zhat
    P[1,:,16] = xhat.^2 .* zhat
    P[2,:,16] = -2*xhat .* yhat .* zhat

    P[2,:,17] = xhat .* yhat
    P[3,:,17] = -xhat .* zhat
    P[2,:,18] = 2*yhat .* zhat
    P[3,:,18] = -zhat.^2
    P[3,:,19] = -2*xhat .* yhat
    P[2,:,20] = xhat .* yhat.^2
    P[3,:,20] = -2*xhat .* yhat .* zhat
    
    P[1,:,21] = -xhat .* yhat
    P[3,:,21] = yhat .* zhat
    P[1,:,22] = -xhat.^2
    P[3,:,22] = 2*xhat .* zhat
    P[1,:,23] = -2*yhat .* zhat
    P[1,:,24] = -2*xhat .* yhat .* zhat
    P[3,:,24] = yhat .* zhat.^2
    
    return P
end

function VondermondeMat()

    # normals
    nl = [0.0;-1.0;0.0]
    nr = [0.0;1.0;0.0]
    nbt = [0.0;0.0;-1.0]
    nt = [0.0;0.0;1.0]
    nf = [1.0;0.0;0.0]
    nbk = [-1.0;0.0;0.0]
    # nodes
    n1 = [-1.;-1.;-1.]
    n2 = [1.;-1.;-1]
    n3 = [1.;1.;-1.]
    n4 = [-1.;1.;-1.]
    n5 = [-1.;-1.;1.]
    n6 = [1.;-1.;1]
    n7 = [1.;1.;1.]
    n8 = [-1.;1.;1.]

    nodes = [n1 n2 n3 n4 n3 n4 n8 n7 n5 n6 n7 n8 n2 n1 n5 n6 n2 n3 n7 n6 n1 n4 n8 n5]
    normals = [nbt nbt nbt nbt nr nr nr nr nt nt nt nt nl nl nl nl nf nf nf nf nbk nbk nbk nbk]
    # vandermonde matrix, V_ij = phi_j(x_i).n_i
    VM = zeros(24,24)
    for i=1:24
        for j=1:24
            P = PrimeBasis([nodes[1,i]], [nodes[2,i]], [nodes[3,i]])
            VM[i,j] = P[1,1,j]*normals[1,i] + P[2,1,j]*normals[2,i] + P[3,1,j]*normals[3,i]
        end
    end

    return VM
end

VondermondeMat (generic function with 1 method)

# Nodal Basis in libCEED
Following gives the nodal basis for quad element that we use in libCEED. Note that we have 8 dofs and since they are vector, we separate its $x,y$ componanats. Note we have not applied `Piola` map on the basis, we should apply it in Qfunction.

In [3]:
VM = VondermondeMat();
invVM = inv(VM);
V = Array{String}(undef, 3, 24)
for i = 1:24
    b = zeros(24)
    b[i] = 1
    xx = invVM * b
    
    vx = Array{String}(undef, 11)
    vy = Array{String}(undef, 11)
    vz = Array{String}(undef, 11)
    
    vx[1] = ""
    vx[2] = "*xhat"
    vx[3] = "*yhat"
    vx[4] = "*zhat"
    # x*y
    vx[5] = "*xhat*yhat"
    # 2*x*y
    vx[6] = "*xhat*yhat"
    # x^2*z
    vx[7] = "*xhat*xhat*zhat"
    # -x*y
    vx[8] = "*xhat*yhat"
    # -x^2
    vx[9] = "*xhat*xhat"
    # -2*y*z
    vx[10] = "*yhat*zhat"
    # -2*x*y*z
    vx[11] = "*xhat*yhat*zhat"
    
    vy[1] = ""
    vy[2] = "*xhat"
    vy[3] = "*yhat"
    vy[4] = "*zhat"
    # -yz
    vy[5] = "*yhat*zhat"
    # -y^2
    vy[6] = "*yhat*yhat"
    # -2*x*z
    vy[7] = "*xhat*zhat"
    # -2*x*y*z
    vy[8] = "*xhat*yhat*zhat"
    # x*y
    vy[9] = "*xhat*yhat"
    # 2*y*z
    vy[10] = "*yhat*zhat"
    # x*y^2
    vy[11] = "*xhat*yhat*yhat"
    
    vz[1] = ""
    vz[2] = "*xhat"
    vz[3] = "*yhat"
    vz[4] = "*zhat"
    # -xz
    vz[5] = "*xhat*zhat"
    # -z^2
    vz[6] = "*zhat*zhat"
    # -2*x*y
    vz[7] = "*xhat*yhat"
    # -2*x*y*z
    vz[8] = "*xhat*yhat*zhat"
    # y*z
    vz[9] = "*yhat*zhat"
    # 2*x*z
    vz[10] = "*xhat*zhat"
    # y*z^2
    vz[11] = "*yhat*zhat*zhat"
    
    V1 = [xx[1] xx[2] xx[3] xx[4] xx[13] 2*xx[14] xx[16] -xx[21] -xx[22] -2*xx[23] -2*xx[24]]
    V2 = [xx[5] xx[6] xx[7] xx[8] -xx[13] -xx[14] -2*xx[15] -2*xx[16] xx[17] 2*xx[18] xx[20]]
    V3 = [xx[9] xx[10] xx[11] xx[12] -xx[17] -xx[18] -2*xx[19] -2*xx[20] xx[21] 2*xx[22] xx[24]]
    VX = join(("$a"vx[i] for (i,a) in enumerate(V1) if a ≠ 0), " + ")
    VY = join(("$a"vy[i] for (i,a) in enumerate(V2) if a ≠ 0), " + ")
    VZ = join(("$a"vz[i] for (i,a) in enumerate(V3) if a ≠ 0), " + ")
    #println("============= Nodal Basis of dof$i =============\n")
    println("Bx[:,$(i)] = @. ",VX, ";")
    println("By[:,$(i)] = @. ",VY, ";")
    println("Bz[:,$(i)] = @. ",VZ, ";")
    #println("\n")
    
    V[1,i] = VX
    V[2,i] = VY
    V[2,i] = VZ
end

Bx[:,1] = @. -0.0625 + -0.125*xhat*yhat + 0.125*xhat*yhat + 0.0625*xhat*xhat;
By[:,1] = @. -0.0625 + 0.0625*xhat + 0.0625*yhat*yhat + -0.0625*xhat*yhat*yhat;
Bz[:,1] = @. -0.125 + 0.125*xhat + 0.125*yhat + 0.125*zhat + -0.125*xhat*yhat + 0.125*xhat*yhat*zhat + -0.125*yhat*zhat + -0.125*xhat*zhat;
Bx[:,2] = @. 0.0625 + -0.125*xhat*yhat + 0.125*xhat*yhat + -0.0625*xhat*xhat;
By[:,2] = @. -0.0625 + -0.0625*xhat + 0.0625*yhat*yhat + 0.0625*xhat*yhat*yhat;
Bz[:,2] = @. -0.125 + -0.125*xhat + 0.125*yhat + 0.125*zhat + 0.125*xhat*yhat + -0.125*xhat*yhat*zhat + -0.125*yhat*zhat + 0.125*xhat*zhat;
Bx[:,3] = @. 0.0625 + 0.125*xhat*yhat + -0.125*xhat*yhat + -0.0625*xhat*xhat;
By[:,3] = @. 0.0625 + 0.0625*xhat + -0.0625*yhat*yhat + -0.0625*xhat*yhat*yhat;
Bz[:,3] = @. -0.125 + -0.125*xhat + -0.125*yhat + 0.125*zhat + -0.125*xhat*yhat + 0.125*xhat*yhat*zhat + 0.125*yhat*zhat + 0.125*xhat*zhat;
Bx[:,4] = @. -0.0625 + 0.125*xhat*yhat + -0.125*xhat*yhat + 0.0625*xhat*xhat;
By[:,4] = @. 0.0625 + -0.062

In [4]:
# this is divergence of the nodal basis that we use it in libCEED
divV = [0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Dhat = divV * invVM

1×24 Matrix{Float64}:
 0.125  0.125  0.125  0.125  0.125  …  0.125  0.125  0.125  0.125  0.125

# CeedBasisView

In [5]:
function GetBasis(xhat, yhat, zhat)

    m = length(xhat)
    Bx = zeros(m,24)
    By = zeros(m,24)
    Bz = zeros(m,24)
    Bx[:,1] = @. -0.0625 + -0.125*xhat*yhat + 0.125*xhat*yhat + 0.0625*xhat*xhat;
    By[:,1] = @. -0.0625 + 0.0625*xhat + 0.0625*yhat*yhat + -0.0625*xhat*yhat*yhat;
    Bz[:,1] = @. -0.125 + 0.125*xhat + 0.125*yhat + 0.125*zhat + -0.125*xhat*yhat + 0.125*xhat*yhat*zhat + -0.125*yhat*zhat + -0.125*xhat*zhat;
    Bx[:,2] = @. 0.0625 + -0.125*xhat*yhat + 0.125*xhat*yhat + -0.0625*xhat*xhat;
    By[:,2] = @. -0.0625 + -0.0625*xhat + 0.0625*yhat*yhat + 0.0625*xhat*yhat*yhat;
    Bz[:,2] = @. -0.125 + -0.125*xhat + 0.125*yhat + 0.125*zhat + 0.125*xhat*yhat + -0.125*xhat*yhat*zhat + -0.125*yhat*zhat + 0.125*xhat*zhat;
    Bx[:,3] = @. 0.0625 + 0.125*xhat*yhat + -0.125*xhat*yhat + -0.0625*xhat*xhat;
    By[:,3] = @. 0.0625 + 0.0625*xhat + -0.0625*yhat*yhat + -0.0625*xhat*yhat*yhat;
    Bz[:,3] = @. -0.125 + -0.125*xhat + -0.125*yhat + 0.125*zhat + -0.125*xhat*yhat + 0.125*xhat*yhat*zhat + 0.125*yhat*zhat + 0.125*xhat*zhat;
    Bx[:,4] = @. -0.0625 + 0.125*xhat*yhat + -0.125*xhat*yhat + 0.0625*xhat*xhat;
    By[:,4] = @. 0.0625 + -0.0625*xhat + -0.0625*yhat*yhat + 0.0625*xhat*yhat*yhat;
    Bz[:,4] = @. -0.125 + 0.125*xhat + -0.125*yhat + 0.125*zhat + 0.125*xhat*yhat + -0.125*xhat*yhat*zhat + 0.125*yhat*zhat + -0.125*xhat*zhat;
    Bx[:,5] = @. 0.0625 + -0.0625*zhat + 0.0625*xhat*xhat*zhat + -0.0625*xhat*xhat;
    By[:,5] = @. 0.125 + 0.125*xhat + 0.125*yhat + -0.125*zhat + -0.125*xhat*zhat + -0.125*xhat*yhat*zhat + 0.125*xhat*yhat + -0.125*yhat*zhat;
    Bz[:,5] = @. -0.0625 + -0.125*xhat*zhat + 0.0625*zhat*zhat + 0.125*xhat*zhat;
    Bx[:,6] = @. -0.0625 + 0.0625*zhat + -0.0625*xhat*xhat*zhat + 0.0625*xhat*xhat;
    By[:,6] = @. 0.125 + -0.125*xhat + 0.125*yhat + -0.125*zhat + 0.125*xhat*zhat + 0.125*xhat*yhat*zhat + -0.125*xhat*yhat + -0.125*yhat*zhat;
    Bz[:,6] = @. -0.0625 + 0.125*xhat*zhat + 0.0625*zhat*zhat + -0.125*xhat*zhat;
    Bx[:,7] = @. -0.0625 + -0.0625*zhat + 0.0625*xhat*xhat*zhat + 0.0625*xhat*xhat;
    By[:,7] = @. 0.125 + -0.125*xhat + 0.125*yhat + 0.125*zhat + -0.125*xhat*zhat + -0.125*xhat*yhat*zhat + -0.125*xhat*yhat + 0.125*yhat*zhat;
    Bz[:,7] = @. 0.0625 + 0.125*xhat*zhat + -0.0625*zhat*zhat + -0.125*xhat*zhat;
    Bx[:,8] = @. 0.0625 + 0.0625*zhat + -0.0625*xhat*xhat*zhat + -0.0625*xhat*xhat;
    By[:,8] = @. 0.125 + 0.125*xhat + 0.125*yhat + 0.125*zhat + 0.125*xhat*zhat + 0.125*xhat*yhat*zhat + 0.125*xhat*yhat + 0.125*yhat*zhat;
    Bz[:,8] = @. 0.0625 + -0.125*xhat*zhat + -0.0625*zhat*zhat + 0.125*xhat*zhat;
    Bx[:,9] = @. -0.0625 + -0.125*xhat*yhat + 0.125*xhat*yhat + 0.0625*xhat*xhat;
    By[:,9] = @. -0.0625 + 0.0625*xhat + 0.0625*yhat*yhat + -0.0625*xhat*yhat*yhat;
    Bz[:,9] = @. 0.125 + -0.125*xhat + -0.125*yhat + 0.125*zhat + 0.125*xhat*yhat + 0.125*xhat*yhat*zhat + -0.125*yhat*zhat + -0.125*xhat*zhat;
    Bx[:,10] = @. 0.0625 + -0.125*xhat*yhat + 0.125*xhat*yhat + -0.0625*xhat*xhat;
    By[:,10] = @. -0.0625 + -0.0625*xhat + 0.0625*yhat*yhat + 0.0625*xhat*yhat*yhat;
    Bz[:,10] = @. 0.125 + 0.125*xhat + -0.125*yhat + 0.125*zhat + -0.125*xhat*yhat + -0.125*xhat*yhat*zhat + -0.125*yhat*zhat + 0.125*xhat*zhat;
    Bx[:,11] = @. 0.0625 + 0.125*xhat*yhat + -0.125*xhat*yhat + -0.0625*xhat*xhat;
    By[:,11] = @. 0.0625 + 0.0625*xhat + -0.0625*yhat*yhat + -0.0625*xhat*yhat*yhat;
    Bz[:,11] = @. 0.125 + 0.125*xhat + 0.125*yhat + 0.125*zhat + 0.125*xhat*yhat + 0.125*xhat*yhat*zhat + 0.125*yhat*zhat + 0.125*xhat*zhat;
    Bx[:,12] = @. -0.0625 + 0.125*xhat*yhat + -0.125*xhat*yhat + 0.0625*xhat*xhat;
    By[:,12] = @. 0.0625 + -0.0625*xhat + -0.0625*yhat*yhat + 0.0625*xhat*yhat*yhat;
    Bz[:,12] = @. 0.125 + -0.125*xhat + 0.125*yhat + 0.125*zhat + -0.125*xhat*yhat + -0.125*xhat*yhat*zhat + 0.125*yhat*zhat + -0.125*xhat*zhat;
    Bx[:,13] = @. 0.0625 + -0.0625*zhat + 0.0625*xhat*xhat*zhat + -0.0625*xhat*xhat;
    By[:,13] = @. -0.125 + -0.125*xhat + 0.125*yhat + 0.125*zhat + 0.125*xhat*zhat + -0.125*xhat*yhat*zhat + 0.125*xhat*yhat + -0.125*yhat*zhat;
    Bz[:,13] = @. -0.0625 + -0.125*xhat*zhat + 0.0625*zhat*zhat + 0.125*xhat*zhat;
    Bx[:,14] = @. -0.0625 + 0.0625*zhat + -0.0625*xhat*xhat*zhat + 0.0625*xhat*xhat;
    By[:,14] = @. -0.125 + 0.125*xhat + 0.125*yhat + 0.125*zhat + -0.125*xhat*zhat + 0.125*xhat*yhat*zhat + -0.125*xhat*yhat + -0.125*yhat*zhat;
    Bz[:,14] = @. -0.0625 + 0.125*xhat*zhat + 0.0625*zhat*zhat + -0.125*xhat*zhat;
    Bx[:,15] = @. -0.0625 + -0.0625*zhat + 0.0625*xhat*xhat*zhat + 0.0625*xhat*xhat;
    By[:,15] = @. -0.125 + 0.125*xhat + 0.125*yhat + -0.125*zhat + 0.125*xhat*zhat + -0.125*xhat*yhat*zhat + -0.125*xhat*yhat + 0.125*yhat*zhat;
    Bz[:,15] = @. 0.0625 + 0.125*xhat*zhat + -0.0625*zhat*zhat + -0.125*xhat*zhat;
    Bx[:,16] = @. 0.0625 + 0.0625*zhat + -0.0625*xhat*xhat*zhat + -0.0625*xhat*xhat;
    By[:,16] = @. -0.125 + -0.125*xhat + 0.125*yhat + -0.125*zhat + -0.125*xhat*zhat + 0.125*xhat*yhat*zhat + 0.125*xhat*yhat + 0.125*yhat*zhat;
    Bz[:,16] = @. 0.0625 + -0.125*xhat*zhat + -0.0625*zhat*zhat + 0.125*xhat*zhat;
    Bx[:,17] = @. 0.125 + 0.125*xhat + -0.125*yhat + -0.125*zhat + -0.125*xhat*yhat + -0.125*xhat*yhat + 0.125*yhat*zhat + 0.125*xhat*yhat*zhat;
    By[:,17] = @. -0.0625 + 0.125*yhat*zhat + 0.0625*yhat*yhat + -0.125*yhat*zhat;
    Bz[:,17] = @. -0.0625 + 0.0625*yhat + 0.0625*zhat*zhat + -0.0625*yhat*zhat*zhat;
    Bx[:,18] = @. 0.125 + 0.125*xhat + 0.125*yhat + -0.125*zhat + -0.125*xhat*yhat + 0.125*xhat*yhat + -0.125*yhat*zhat + -0.125*xhat*yhat*zhat;
    By[:,18] = @. 0.0625 + 0.125*yhat*zhat + -0.0625*yhat*yhat + -0.125*yhat*zhat;
    Bz[:,18] = @. -0.0625 + -0.0625*yhat + 0.0625*zhat*zhat + 0.0625*yhat*zhat*zhat;
    Bx[:,19] = @. 0.125 + 0.125*xhat + 0.125*yhat + 0.125*zhat + 0.125*xhat*yhat + 0.125*xhat*yhat + 0.125*yhat*zhat + 0.125*xhat*yhat*zhat;
    By[:,19] = @. 0.0625 + -0.125*yhat*zhat + -0.0625*yhat*yhat + 0.125*yhat*zhat;
    Bz[:,19] = @. 0.0625 + 0.0625*yhat + -0.0625*zhat*zhat + -0.0625*yhat*zhat*zhat;
    Bx[:,20] = @. 0.125 + 0.125*xhat + -0.125*yhat + 0.125*zhat + 0.125*xhat*yhat + -0.125*xhat*yhat + -0.125*yhat*zhat + -0.125*xhat*yhat*zhat;
    By[:,20] = @. -0.0625 + -0.125*yhat*zhat + 0.0625*yhat*yhat + 0.125*yhat*zhat;
    Bz[:,20] = @. 0.0625 + -0.0625*yhat + -0.0625*zhat*zhat + 0.0625*yhat*zhat*zhat;
    Bx[:,21] = @. -0.125 + 0.125*xhat + 0.125*yhat + 0.125*zhat + -0.125*xhat*yhat + -0.125*xhat*yhat + -0.125*yhat*zhat + 0.125*xhat*yhat*zhat;
    By[:,21] = @. -0.0625 + 0.125*yhat*zhat + 0.0625*yhat*yhat + -0.125*yhat*zhat;
    Bz[:,21] = @. -0.0625 + 0.0625*yhat + 0.0625*zhat*zhat + -0.0625*yhat*zhat*zhat;
    Bx[:,22] = @. -0.125 + 0.125*xhat + -0.125*yhat + 0.125*zhat + -0.125*xhat*yhat + 0.125*xhat*yhat + 0.125*yhat*zhat + -0.125*xhat*yhat*zhat;
    By[:,22] = @. 0.0625 + 0.125*yhat*zhat + -0.0625*yhat*yhat + -0.125*yhat*zhat;
    Bz[:,22] = @. -0.0625 + -0.0625*yhat + 0.0625*zhat*zhat + 0.0625*yhat*zhat*zhat;
    Bx[:,23] = @. -0.125 + 0.125*xhat + -0.125*yhat + -0.125*zhat + 0.125*xhat*yhat + 0.125*xhat*yhat + -0.125*yhat*zhat + 0.125*xhat*yhat*zhat;
    By[:,23] = @. 0.0625 + -0.125*yhat*zhat + -0.0625*yhat*yhat + 0.125*yhat*zhat;
    Bz[:,23] = @. 0.0625 + 0.0625*yhat + -0.0625*zhat*zhat + -0.0625*yhat*zhat*zhat;
    Bx[:,24] = @. -0.125 + 0.125*xhat + 0.125*yhat + -0.125*zhat + 0.125*xhat*yhat + -0.125*xhat*yhat + 0.125*yhat*zhat + -0.125*xhat*yhat*zhat;
    By[:,24] = @. -0.0625 + -0.125*yhat*zhat + 0.0625*yhat*yhat + 0.125*yhat*zhat;
    Bz[:,24] = @. 0.0625 + -0.0625*yhat + -0.0625*zhat*zhat + 0.0625*yhat*zhat*zhat;

    B = zeros(3m, 24)
    B[1:m,:] = Bx[1:m,:]
    B[m+1:2*m,:] = By[1:m,:]
    B[2*m+1:3*m,:] = Bz[1:m,:]
    Dhat = zeros(1,24)
    Dhat[1,:] .= 0.125;
    Div = repeat(Dhat, inner=(m,1))
    
    return B, Div
end

GetBasis (generic function with 1 method)

In [6]:
function GetQuadrature3D(Q, quad_mode)
    """ 
    Input:
    Q: number of quadrature points in 1D over [-1,1]
    quad_mode: GAUSS or LOBATTO
    Return:Gauss Quadrature data over [-1,1]^3.
    w: weights of quadrature pts
    qx: quadrature pts in x
    qy: quadrature pts in y
    qz
    """
    
    # 1D Gauss
    if quad_mode == "GAUSS"
        q = zgj(Q, 0.0, 0.0)
        w1 = wgj(q, 0.0, 0.0)
    elseif quad_mode == "LOBATTO"
        q = zglj(Q, 0.0, 0.0)
        w1 = wglj(q, 0.0, 0.0)
    end
    w = zeros(Q*Q*Q)
    qx = zeros(Q*Q*Q)
    qy = zeros(Q*Q*Q)
    qz = zeros(Q*Q*Q)
    for l=1:Q
        for i=1:Q
            for j=1:Q
                k = Q*Q*(l-1) + (i-1)*Q +j
                qx[k] = q[j]
                qy[k] = q[i]
                qz[k] = q[l]
                w[k] = w1[j]*w1[i]*w1[l]
            end
        end
    end
    return w, qx, qy, qz
end

GetQuadrature3D (generic function with 1 method)

In [10]:
Q = 3
num_qpts = Q*Q*Q
w, qx, qy, qz = GetQuadrature3D(Q, "GAUSS")
B, Div = GetBasis(qx, qy, qz);

In [11]:
Div

27×24 Matrix{Float64}:
 0.125  0.125  0.125  0.125  0.125  …  0.125  0.125  0.125  0.125  0.125
 0.125  0.125  0.125  0.125  0.125     0.125  0.125  0.125  0.125  0.125
 0.125  0.125  0.125  0.125  0.125     0.125  0.125  0.125  0.125  0.125
 0.125  0.125  0.125  0.125  0.125     0.125  0.125  0.125  0.125  0.125
 0.125  0.125  0.125  0.125  0.125     0.125  0.125  0.125  0.125  0.125
 0.125  0.125  0.125  0.125  0.125  …  0.125  0.125  0.125  0.125  0.125
 0.125  0.125  0.125  0.125  0.125     0.125  0.125  0.125  0.125  0.125
 0.125  0.125  0.125  0.125  0.125     0.125  0.125  0.125  0.125  0.125
 0.125  0.125  0.125  0.125  0.125     0.125  0.125  0.125  0.125  0.125
 0.125  0.125  0.125  0.125  0.125     0.125  0.125  0.125  0.125  0.125
 0.125  0.125  0.125  0.125  0.125  …  0.125  0.125  0.125  0.125  0.125
 0.125  0.125  0.125  0.125  0.125     0.125  0.125  0.125  0.125  0.125
 0.125  0.125  0.125  0.125  0.125     0.125  0.125  0.125  0.125  0.125
 ⋮                          

In [12]:
B

81×24 Matrix{Float64}:
 -0.025        0.025        0.025       …  -0.0112702   -0.0887298
 -0.0625       0.0625       0.0625         -0.00635083  -0.05
 -0.025        0.025        0.025          -0.0014315   -0.0112702
 -0.025        0.025        0.025          -0.125       -0.125
 -0.0625       0.0625       0.0625         -0.0281754   -0.0281754
 -0.025        0.025        0.025       …   0.0686492    0.0686492
 -0.025        0.025        0.025          -0.23873     -0.16127
 -0.0625       0.0625       0.0625         -0.05        -0.00635083
 -0.025        0.025        0.025           0.13873      0.148569
 -0.025        0.025        0.025           0.025       -0.318649
 -0.0625       0.0625       0.0625      …  -0.0281754   -0.221825
 -0.025        0.025        0.025          -0.0813508   -0.125
 -0.025        0.025        0.025          -0.221825    -0.221825
  ⋮                                     ⋱               
 -0.05        -0.00635083  -0.05            0.110912     0.0140877
